# Hands On Machine Learning Chapter 9
Examples from HOML on using TensorFlow

In [1]:
import tensorflow as tf
import os
from datetime import datetime

In [2]:
# Define the graph
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [3]:
# Add a node to intialise all variables
init = tf.global_variables_initializer()

In [4]:
# Create a session and evaluate f
with tf.Session() as sess:
    sess.run(init)
    result = f.eval()
print(result)

42


In [5]:
# Clear the graph
tf.reset_default_graph()

In [6]:
# Linear regression
import numpy as np
from sklearn.datasets import fetch_california_housing

In [7]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [8]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaler.fit_transform(housing.data)]
# print(scaled_housing_data_plus_bias[:10, :])

In [10]:
# Define the computation graph
# Use scaled for comparison with gradient descent
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')  
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y' )
XT = tf.transpose(X)
# Compute model parameters using Normal Equation
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [11]:
with tf.Session() as sess:
    theta_value = theta.eval()

print(theta_value)

[[ 2.06856298]
 [ 0.82961965]
 [ 0.11875178]
 [-0.26552707]
 [ 0.30569667]
 [-0.00450281]
 [-0.03932635]
 [-0.8998825 ]
 [-0.87053877]]


In [12]:
tf.reset_default_graph()

In [13]:
# Linear regression using gradient descent
n_epochs = 2000
learning_rate = 0.01

In [14]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
# gradients = 2/m * tf.matmul(tf.transpose(X), error)  # Manually derived gradient
# gradients = tf.gradients(mse, [theta])[0]  # Autodiff
# training_op = tf.assign(theta, theta - learning_rate * gradients)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [15]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % (n_epochs / 10) == 0:
            print('Epoch {epoch}  MSE = {mse}'.format(epoch=epoch, mse=mse.eval()))
        sess.run(training_op)
    best_theta = theta.eval()
print(best_theta)

Epoch 0  MSE = 6.87245130539
Epoch 200  MSE = 0.646441102028
Epoch 400  MSE = 0.540355324745
Epoch 600  MSE = 0.529172837734
Epoch 800  MSE = 0.525422751904
Epoch 1000  MSE = 0.524504840374
Epoch 1200  MSE = 0.524343013763
Epoch 1400  MSE = 0.524322807789
Epoch 1600  MSE = 0.524320721626
Epoch 1800  MSE = 0.524321079254
[[ 2.06855607]
 [ 0.82962835]
 [ 0.11875465]
 [-0.26554158]
 [ 0.30570725]
 [-0.00450202]
 [-0.03935729]
 [-0.89985526]
 [-0.87051153]]


In [19]:
tf.reset_default_graph()
root_logdir = os.path.join(os.environ.get('MDA_DATA_DIR'), 'tf_logs')
now = datetime.utcnow().strftime('%Y%m%dT%H%M%S')
logdir = os.path.join(root_logdir, 'run-' + now)
# Minibatch training
X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
# gradients = 2/m * tf.matmul(tf.transpose(X), error)  # Manually derived gradient
# gradients = tf.gradients(mse, [theta])[0]  # Autodiff
# training_op = tf.assign(theta, theta - learning_rate * gradients)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [20]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % (n_epochs / 10) == 0:
            print('Epoch {epoch}'.format(epoch=epoch))
        for batch in range(n_batches):
            X_batch = scaled_housing_data_plus_bias[batch_size*batch:min(m, batch_size*(batch + 1)), :]
            y_batch = housing.target.reshape(-1, 1)[batch_size*batch:min(m, batch_size*(batch + 1))]
            if batch % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
                step = epoch * n_batches + batch
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={
                X: X_batch,
                y: y_batch
            })
    best_theta = theta.eval()
print(best_theta)

Epoch 0
